In [1]:
import requests
import json
import numpy as np
import joblib

X_test_unscaled = np.load('../data/processed_multiclass/X_test_unscaled.npy')
y_test = np.load('../data/processed_multiclass/y_test.npy')

preprocessor = joblib.load('../models/preprocessor_multiclass.pkl')
feature_columns = preprocessor['feature_columns']
classes = preprocessor['classes']

print("Testing API with different attack types:\n")

for attack_idx in [0, 2, 4, 10, 12]:
    sample_indices = np.where(y_test == attack_idx)[0]
    if len(sample_indices) > 0:
        sample = X_test_unscaled[sample_indices[0]]
        features_dict = {name: float(value) for name, value in zip(feature_columns, sample)}
        
        response = requests.post(
            "http://localhost:8000/predict/full",
            json={"features": features_dict}
        )
        
        result = response.json()
        actual = classes[attack_idx]
        
        print(f"Actual: {actual:<30} → Predicted: {result['prediction']}")
        print(f"  Confidence: {result['confidence']*100:.1f}%")
        print(f"  Top 3: {list(result['top_3_predictions'].keys())}")
        if result.get('warning'):
            print(f"  {result['warning']}")
        print()

Testing API with different attack types:

Actual: BENIGN                         → Predicted: BENIGN
  Confidence: 100.0%
  Top 3: ['BENIGN', 'Web Attack - XSS', 'Web Attack - Sql Injection']

Actual: DDoS                           → Predicted: DDoS
  Confidence: 100.0%
  Top 3: ['DDoS', 'BENIGN', 'DoS Hulk']

Actual: DoS Hulk                       → Predicted: DoS Hulk
  Confidence: 100.0%
  Top 3: ['DoS Hulk', 'Web Attack - XSS', 'Web Attack - Sql Injection']

Actual: PortScan                       → Predicted: PortScan
  Confidence: 99.9%
  Top 3: ['PortScan', 'BENIGN', 'Web Attack - XSS']

Actual: Web Attack - Brute Force       → Predicted: Web Attack - Brute Force
  Confidence: 79.4%
  Top 3: ['Web Attack - Brute Force', 'Web Attack - XSS', 'BENIGN']



In [3]:
import pandas as pd
import numpy as np
import joblib

X_test_unscaled = np.load('../data/processed_multiclass/X_test_unscaled.npy')
y_test = np.load('../data/processed_multiclass/y_test.npy')

preprocessor = joblib.load('../models/preprocessor_multiclass.pkl')
feature_columns = preprocessor['feature_columns']
classes = preprocessor['classes']

# 5 samples of each major attack type
attack_types_to_include = [0, 2, 4, 10, 7, 11, 12]  # BENIGN, DDoS, DoS Hulk, PortScan, FTP-Patator, SSH-Patator, Web Attack
samples_per_type = 5

selected_samples = []
selected_labels = []

for attack_type in attack_types_to_include:
    indices = np.where(y_test == attack_type)[0][:samples_per_type]
    selected_samples.extend(X_test_unscaled[indices])
    selected_labels.extend(y_test[indices])

df = pd.DataFrame(selected_samples, columns=feature_columns)
df['Actual_Label'] = [classes[label] for label in selected_labels]

df.to_csv('../test_data_multiclass.csv', index=False)

print(f" Created test_data_multiclass.csv")
print(f"  Total flows: {len(df)}")
print(f"\nAttack types included:")
print(df['Actual_Label'].value_counts())

 Created test_data_multiclass.csv
  Total flows: 35

Attack types included:
Actual_Label
BENIGN                      5
DDoS                        5
DoS Hulk                    5
PortScan                    5
FTP-Patator                 5
SSH-Patator                 5
Web Attack - Brute Force    5
Name: count, dtype: int64
